# Regression TensorFlow

### Install Requirements

In [1]:
! pip install memory_profiler

    100% |████████████████████████████████| 40kB 5.1MB/s 
  Stored in directory: /root/.cache/pip/wheels/f0/ff/63/fdbff3f1e1b76ad4eae491dd5b190902906b093e93eb86dd5a
Successfully built memory-profiler


In [0]:
%load_ext memory_profiler

### Import Libraries

In [0]:
from IPython import display
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
from tensorflow.python.data import Dataset
from tensorflow.estimator.inputs import numpy_input_fn, pandas_input_fn
from tensorflow.feature_column import numeric_column
from sklearn.preprocessing import MinMaxScaler

tf.logging.set_verbosity(tf.logging.WARN)
pd.options.display.max_rows = 10
pd.options.display.float_format = '{:.2f}'.format

# Pretty Display of Variables
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

### Necessary Functions

In [0]:
def numeric_features(feature_columns):
    """Construct the TensorFlow Feature Columns.

    Args:
        data(List): The names of the numerical input features to use.
    ....
    Returns:
        A set of feature columns
    """ 
  
    return set([numeric_column(feature) for feature in feature_columns])

In [0]:
def pandas_fn(X, Y, features=None, batch_size=1, shuffle=True, num_epochs=None):
    """ Creates a input function that constructs the input function
    for the model. You can pass the data after splitting the training and 
    test data. Function selects features while preparing the data.
  
    Args:
        X: Pandas DataFrame containing the training data.
        Y: Pandas Series containing the labels
        features: A List containing the names of the features. Default to None
        batch_size: Size of each features to fetch. Default value `1`
        shuffle: Shuffle the data. Default value `True`
        num_epochs: Iteration to fetch the data. `None` for infinity.

    Returns:
        pandas input function
    """
    if features is not None:
        return pandas_input_fn(X[features], Y, batch_size=batch_size, shuffle=shuffle, num_epochs=num_epochs)
    else:
        return pandas_input_fn(X, Y, batch_size=batch_size, shuffle=shuffle, num_epochs=num_epochs)

In [0]:
def numpy_fn(X, Y, features=None, batch_size=1, shuffle=True, num_epochs=None):
    """ Creates a input function that constructs the input function
    for the model. You can pass the data after splitting the training and 
    test data. Function selects features while preparing the data.

    Args:
        X: Pandas DataFrame containing the training data.
        Y: Pandas Series or DataFrame containing the labels
        features: A List containing the names of the features. Default to None.
        batch_size: Size of each features to fetch. Default value `1`
        shuffle: Shuffle the data. Default value `True`
        num_epochs: Iteration to fetch the data. `None` for infinity.
    """
  
    if features is not None:
        return numpy_input_fn(x={key:np.array(value) for key,value in dict(X[features]).items()},
                        y=Y.values, batch_size=batch_size, shuffle=shuffle, num_epochs=num_epochs)
    else:
        return numpy_input_fn(x={key:np.array(value) for key,value in dict(X).items()},
                        y=Y.values, batch_size=batch_size, shuffle=shuffle, num_epochs=num_epochs)

In [0]:
def compute_predictions(model, predict_fn):
    """ A Function that produces resuts for the given model.

    Args:
        model: A model data was train on. tf.estimator object
        predict_fn: A function used to predict the output

    Returns:
        np.array containing prediction results
    """
    predictions = model.predict(input_fn=predict_fn)
    return np.array([item['predictions'][0] for item in predictions])

In [0]:
def loss(predictions, y):
    """ Loss function to find the error value of our model. Mean squared error.
  
    Args:
        predictions: Predicted values
        y: Actual values
  
    Returns:
        A Tensor object to containing the error value.
  
    """
    return tf.reduce_mean(tf.squared_difference(predictions, y.values))

In [0]:
def loss2(predictions, y):
    """ Loss function to find the error value of our model
  
    Args:
        predictions: Predicted values
        y: Actual values
  
    Returns:
        A Tensor object to containing the error value.
  
  """
    return tf.reduce_mean(tf.squared_difference(predictions, y))

In [ ]:
def process_california_data(ratio=0.8):
    """ Downloads the california housing dataset, preprocess the data and splits it.
    Args:
        ratio: Split ratio. Default is 0.8
    
    """

    print('Downloading data from web..')
    # Load training & test data set.
    train_df = pd.read_csv("https://download.mlcc.google.com/mledu-datasets/california_housing_train.csv", sep=",")
    test_df = pd.read_csv("https://download.mlcc.google.com/mledu-datasets/california_housing_test.csv", sep=",")

    print('Merging data..')
    # Merge data
    df = pd.concat([train_df, test_df])
    
    # Drop NA values
    df.dropna(inplace=True)
    
    print('Creating a synthetic feature..')
    # Create a synthetic feature.
    df["rooms_per_person"] = df["total_rooms"] / df["population"]
    
    # Scale the huge value columns
    df["median_house_value"] = df["median_house_value"] / 1000 

    print('Shuffling the data..')
    # Shuffle
    df = df.reindex(np.random.permutation(df.index))

    print('Heatmap of our data..')
    sample = df.sample(n=500)
    sns.heatmap(
        sample.corr(), 
        xticklabels=sample.columns.values,
        yticklabels=sample.columns.values,
        fmt=".2f",
        annot=True
    )

    print('Scaling the data between 0-1..')
    # Scale the data between 0-1
    scaler = MinMaxScaler(feature_range=(0,1))
    scale_columns = list(df.columns[2:])
    scaled = df.copy()
    scaled[scale_columns] = scaler.fit_transform(df[scale_columns])
    print("Data was multiplied by {:.6f} and added {:.4f}".format(scaler.scale_[0], scaler.min_[0]))


    print('Splitting train & test frames..')
    # Split
    
    split_size_head = round(ratio*scaled.shape[0])
    split_size_tail = scaled.shape[0] - split_size_head
    train = scaled.head(split_size_head)
    test = scaled.tail(split_size_tail)

    return train, test, scaler

In [ ]:
train, test, scaler = process_california_data()

### Model Functions

#### Estimator Functions

In [0]:
def linear_estimator(labels, funcs, feature_columns, learning_rate, steps, quiet=False):
    """ Linear Regression using high level Estimator API
  
    Args:
        labels: A dictionary contains labels as pandas Series for training and testing. 
                {train: y_train, test: y_test}
        funcs: A dictionary contains training/test/predict function to hook up with model.
                {train: train_fn, test: test_fn, predict: predict_fn}
        feature_columns: Feature columns as list
        learning_rate: Learning rate of the model as float
        steps: Number of steps required to run the model with parameters as integer
        quiet: Selects whether to print results or not.
    
    """
    Y_train, Y_test = labels['train'], labels['test']
    train_fn, test_fn, predict_fn = funcs['train'], funcs['test'], funcs['predict']
    
    # Model parameters
    model = tf.estimator.LinearRegressor(feature_columns=numeric_features(feature_columns), 
                                         optimizer=tf.train.AdamOptimizer(learning_rate))
      
    # Train the model, starting from the prior state.
    model.train(input_fn=train_fn, max_steps=steps)

    # Evalute model
    eval_result = model.evaluate(input_fn=test_fn)
    
    if quiet is False:
        print("Printing Eval Results....\n", eval_result)

    # Get the loss value
    mse = eval_result["average_loss"]

    # Compute predictions.
    predictions = compute_predictions(model, predict_fn)     

    # Compute loss.
    test_loss = loss(predictions, Y_test)
    
    with tf.Session() as sess:
        test_mse = test_loss.eval()

    if quiet is False:
        # Print the current loss.
        print("  Loss for Train: %0.6f" % (mse**0.5))
        print("  Loss for Test: %0.6f" % (test_mse**0.5))

In [0]:
def neural_estimator(labels, funcs, feature_columns, learning_rate, steps, hidden_units, quiet=False):
    """ Deep Neural Networks
  
    Args:
        labels: A dictionary contains labels as pandas Series for training and testing. 
                {train: y_train, test: y_test}
        funcs: A dictionary contains training/test/predict function to hook up with model.
                {train: train_fn, test: test_fn, predict: predict_fn}
        feature_columns: Feature columns as list
        learning_rate: Learning rate of the model as float
        steps: Number of steps required to run the model with parameters as integer
        hidden_units: DNN layer sizes
        quiet: Selects whether to print results or not.
        
    """
    Y_train, Y_test = labels['train'], labels['test']
    train_fn, test_fn, predict_fn = funcs['train'], funcs['test'], funcs['predict']
 
    # Create a DNNRegressor object
    model = tf.estimator.DNNRegressor(
          feature_columns=numeric_features(feature_columns),
          hidden_units=hidden_units,
          optimizer=tf.train.AdamOptimizer(learning_rate)
    )
      
    # Train the model, starting from the prior state.
    model.train(input_fn=train_fn, max_steps=steps)
  
    # Evalute model
    eval_result = model.evaluate(input_fn=test_fn)
  
    if quiet is False:
        print("Printing Eval Results....\n", eval_result)
  
    # Get the loss value
    mse = eval_result["average_loss"]

    # Compute predictions.
    predictions = compute_predictions(model, predict_fn)     

    # Compute loss
    test_loss = loss(predictions, Y_test)

    with tf.Session() as sess:
        test_mse = test_loss.eval()

    if quiet is False:
        # Print the current loss.
        print("  Loss for Train: %0.6f" % (mse**0.5))
        print("  Loss for Test: %0.6f" % (test_mse**0.5))

#### Low Level Functions

In [0]:
def linear_reg(train, test, feature_size, epochs=500, learning_rate=0.1):
    """ Linear Regression with one feature using low level API 
  
    Args:
        train: A dictionary contains train features and labels as np.arrays
                {x: X_train, y: Y_train}
        test: A dictionary contains train features and labels as np.arrays
                {x: X_test, y: Y_test}
        feature_size: Indicates how many feature will be used in model
        epochs: How many times should neural network run
        learning_rate: Learning rate of the model
  
    """
    X_train, Y_train = train['x'], train['y'] 
    X_test, Y_test = test['x'], test['y']
  
    # Input layer
    with tf.variable_scope("input"):
        X = tf.placeholder(tf.float32, shape=(None, feature_size))
        w = tf.Variable(tf.random_normal([feature_size, 1]))
        b = tf.Variable(tf.random_normal([]))
    
    # Linear Model
    if feature_size == 1: 
        with tf.variable_scope("linear_model"):
            model = w * X + b
    else: 
        with tf.variable_scope("linear_model"):
            model = tf.add(tf.matmul(X, w), b)
  
    # Cost Function
    with tf.variable_scope("cost"):
        Y = tf.placeholder(tf.float32, shape=(None, 1))
        cost = tf.losses.mean_squared_error(Y, model)
  
    # Optimizer Function
    with tf.variable_scope("train"):
        opt = tf.train.AdamOptimizer(learning_rate).minimize(cost)
    
    
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
    
        for epoch in range(epochs):
        # Feed in the training data and do one step of neural network training
            sess.run(opt, feed_dict={X:X_train, Y:Y_train})
      
        mse = sess.run(cost, feed_dict={X: X_train, Y: Y_train})
        test_mse = sess.run(cost, feed_dict={X: X_test, Y: Y_test})
        
        # Print the current loss.
        print("  Loss for Train: %0.6f" % (mse**0.5))
        print("  Loss for Test: %0.6f" % (test_mse**0.5))

In [0]:
def neural_network(train, test, feature_size, layer_sizes, epochs=500, output_size=1, learning_rate=0.1):
    """ Neural network using low level API
  
    Args:
        train: A dictionary contains train features and labels as np.arrays
                {x: X_train, y: Y_train}
        test: A dictionary contains train features and labels as np.arrays
                {x: X_test, y: Y_test}
        feature_size: Size of the features
        output_size: Neural network output size.
        epochs: How many times should neural network run
        learning_rate: Learning rate of the model
        layer_sizes: A list containing layer sizes. [l1, l2, l3] 
  
    """
    X_train, Y_train = train['x'], train['y'] 
    X_test, Y_test = test['x'], test['y']
  
    layer_1_nodes = layer_sizes[0]
    layer_2_nodes = layer_sizes[1]
    layer_3_nodes = layer_sizes[2]
  
    with tf.variable_scope("input"):   # Input layer
        X = tf.placeholder(tf.float32, shape=(None, feature_size))
    
    with tf.variable_scope("layer_1"):  # Layer 1
        weights = tf.get_variable(name="weights1", shape=(feature_size, layer_1_nodes), 
                              initializer=tf.contrib.layers.xavier_initializer())
        biases = tf.get_variable(name="biases1", shape=(layer_1_nodes),
                            initializer=tf.zeros_initializer())
        layer_1_output = tf.nn.relu_layer(X, weights, biases)
  
    
    with tf.variable_scope("layer_2"):   # Layer 2
        weights = tf.get_variable(name="weights2", shape=(layer_1_nodes, layer_2_nodes), 
                              initializer=tf.contrib.layers.xavier_initializer())
    
        biases = tf.get_variable(name="biases2", shape=(layer_2_nodes),
                            initializer=tf.zeros_initializer())
    
        layer_2_output = tf.nn.relu_layer(layer_1_output, weights, biases)
    
    with tf.variable_scope("layer_3"):   # Layer 3
        weights = tf.get_variable(name="weights3", shape=(layer_2_nodes, layer_3_nodes), 
                              initializer=tf.contrib.layers.xavier_initializer())
        biases = tf.get_variable(name="biases3", shape=(layer_3_nodes),
                            initializer=tf.zeros_initializer())
        layer_3_output = tf.nn.relu_layer(layer_2_output, weights, biases)
  
  
    with tf.variable_scope("output"):   # Output Layer
        weights = tf.get_variable(name="weights", shape=(layer_3_nodes, output_size), 
                              initializer=tf.contrib.layers.xavier_initializer())  
        biases = tf.get_variable(name="biases", shape=(output_size),
                            initializer=tf.zeros_initializer())
        predictions = tf.nn.relu_layer(layer_3_output, weights, biases)
  
  
    with tf.variable_scope("cost"):
        Y = tf.placeholder(tf.float32, shape=(None, 1))
        cost = loss2(predictions, Y)
    
    with tf.variable_scope("train"):
        opt = tf.train.AdamOptimizer(learning_rate).minimize(cost)
    
    
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
    
        for epoch in range(epochs):
            # Feed in the training data and do one step of neural network training
            sess.run(opt, feed_dict={X:X_train, Y:Y_train})
      
            mse = sess.run(cost, feed_dict={X: X_train, Y: Y_train})
            test_mse = sess.run(cost, feed_dict={X: X_test, Y: Y_test})

            # Print the current loss.
            print("  Loss for Train: %0.6f" % (mse**0.5))
            print("  Loss for Test: %0.6f" % (test_mse**0.5))

### Linear Regression High Level API

**Model Parameters for Linear Model**

In [0]:
# Model Parameters
learning_rate = 0.02
steps = 200
batch_size = 100

**Linear Regression with One Variable**

In [0]:
feature_columns = ["median_income"]
label = "median_house_value"

# Input functions
train_input = pandas_fn(train, train[label], features=feature_columns, batch_size=batch_size)
test_input = pandas_fn(train, train[label], features=feature_columns, batch_size=1, shuffle=False, num_epochs=1)
predict_input = pandas_fn(test, test[label], features=feature_columns, batch_size=1, shuffle=False, num_epochs=1)

fns = {'train': train_input, 'test': test_input, 'predict': predict_input}
fn_labels = {'train': train[label], 'test': test[label]}

In [26]:
linear_estimator(fn_labels, fns, feature_columns, learning_rate, steps)

Printing Eval Results....
 {'average_loss': 0.030024657, 'label/mean': 0.3967738, 'loss': 0.030024657, 'prediction/mean': 0.39747584, 'global_step': 200}
  Loss for Train: 0.173276
  Loss for Test: 0.174853


In [28]:
%memit linear_estimator(fn_labels, fns, feature_columns, learning_rate, steps, quiet=True)

Printing Eval Results....
 {'average_loss': 0.029991103, 'label/mean': 0.3967738, 'loss': 0.029991103, 'prediction/mean': 0.3928956, 'global_step': 200}
  Loss for Train: 0.173179
  Loss for Test: 0.174810
peak memory: 886.41 MiB, increment: 2.91 MiB


In [29]:
%time linear_estimator(fn_labels, fns, feature_columns, learning_rate, steps, quiet=True)

Printing Eval Results....
 {'average_loss': 0.030032124, 'label/mean': 0.3967738, 'loss': 0.030032124, 'prediction/mean': 0.39253742, 'global_step': 200}
  Loss for Train: 0.173298
  Loss for Test: 0.174899
CPU times: user 1min 3s, sys: 8.69 s, total: 1min 12s
Wall time: 43.6 s


In [30]:
%prun linear_estimator(fn_labels, fns, feature_columns, learning_rate, steps, quiet=True)

Printing Eval Results....
 {'average_loss': 0.030006127, 'label/mean': 0.3967738, 'loss': 0.030006127, 'prediction/mean': 0.39616334, 'global_step': 200}
  Loss for Train: 0.173223
  Loss for Test: 0.174818
 

**Results**

**Train Loss (RMSE)**: 0.173223  
**Validation Loss(RMSE)**: 0.174818


**%time**    
CPU times: user 1min 3s, sys: 8.69 s, total: 1min 12s
Wall time: 43.6 s

**%prun**   
6534300 function calls (6256122 primitive calls) in 47.238 seconds

**%memit**   
peak memory: 886.41 MiB, increment: 2.91 MiB


**Model Parameters**  
learning_rate =  0.02
steps = 200
batch_size = 100 
periods = 10  


**Linear Regression with Multiple Variables**

In [0]:
feature_columns = ["median_income", "rooms_per_person", "total_rooms", "housing_median_age"]
label = "median_house_value"

# Input functions
train_input = pandas_fn(train, train[label], features=feature_columns, batch_size=batch_size)
test_input = pandas_fn(train, train[label], features=feature_columns, batch_size=1, shuffle=False, num_epochs=1)
predict_input = pandas_fn(test, test[label], features=feature_columns, batch_size=1, shuffle=False, num_epochs=1)

fns = {'train': train_input, 'test': test_input, 'predict': predict_input}
fn_labels = {'train': train[label], 'test': test[label]}

In [ ]:
linear_estimator(fn_labels, fns, feature_columns, learning_rate, steps)

In [32]:
%memit linear_estimator(fn_labels, fns, feature_columns, learning_rate, steps, quiet=True)

Printing Eval Results....
 {'average_loss': 0.027768757, 'label/mean': 0.3967738, 'loss': 0.027768757, 'prediction/mean': 0.38962597, 'global_step': 200}
  Loss for Train: 0.166640
  Loss for Test: 0.167832
peak memory: 895.79 MiB, increment: 7.20 MiB


In [33]:
%time linear_estimator(fn_labels, fns, feature_columns, learning_rate, steps, quiet=True)

Printing Eval Results....
 {'average_loss': 0.02755624, 'label/mean': 0.3967738, 'loss': 0.02755624, 'prediction/mean': 0.3990464, 'global_step': 200}
  Loss for Train: 0.166001
  Loss for Test: 0.167204
CPU times: user 1min 20s, sys: 11.1 s, total: 1min 31s
Wall time: 55.2 s


In [34]:
%prun linear_estimator(fn_labels, fns, feature_columns, learning_rate, steps, quiet=True)

Printing Eval Results....
 {'average_loss': 0.027723497, 'label/mean': 0.3967738, 'loss': 0.027723497, 'prediction/mean': 0.38969576, 'global_step': 200}
  Loss for Train: 0.166504
  Loss for Test: 0.167672
 

**Results**

**Train Loss (RMSE)**: 0.166640  
**Validation Loss(RMSE)**: 0.167832

**%time**    
CPU times: user 1min 20s, sys: 11.1 s, total: 1min 31s
Wall time: 55.2 s

**%prun**   
7201219 function calls (6898102 primitive calls) in 58.285 seconds

**%memit**   
peak memory: 895.79 MiB, increment: 7.20 MiB

**Model Parameters**  
learning_rate = 0.02
steps = 200  
batch_size = 100  
periods = 10  



### Linear Regression Low Level API

**Model Parameters for Linear Model**

In [0]:
# Model Parameters
learning_rate = 0.02
steps = 200
# batch_size = 100

**Linear Regression with One Variable**

In [0]:
feature_columns = "median_income"
label = "median_house_value"

In [0]:
X_train = train[feature_columns].values.reshape(-1,1)
Y_train = train[label].values.reshape(-1,1)
X_test = test[feature_columns].values.reshape(-1,1)
Y_test = test[label].values.reshape(-1,1)

_train = {'x': X_train, 'y': Y_train}
_test = {'x': X_test, 'y': Y_test}

In [ ]:
linear_reg(_train, _test, feature_size=1, epochs=steps, learning_rate=learning_rate)

In [44]:
%memit linear_reg(_train, _test, feature_size=1, epochs=steps, learning_rate=learning_rate)

Training Cost Final: 0.17803674695014315  Testing Cost Final: 0.18174758961087978
peak memory: 912.58 MiB, increment: 0.41 MiB


In [45]:
%time linear_reg(_train, _test, feature_size=1, epochs=steps, learning_rate=learning_rate)

Training Cost Final: 0.17281952792806496  Testing Cost Final: 0.1745345569816395
CPU times: user 570 ms, sys: 43 ms, total: 613 ms
Wall time: 533 ms


In [46]:
%prun linear_reg(_train, _test, feature_size=1, epochs=steps, learning_rate=learning_rate)

Training Cost Final: 0.17222949879687607  Testing Cost Final: 0.17438799993362555
 

**Results**

**Train Loss (RMSE)**: 0.172229  
**Validation Loss(RMSE)**: 0.174387

**%time**    
CPU times: user 570 ms, sys: 43 ms, total: 613 ms
Wall time: 533 ms

**%prun**   
258450 function calls (252521 primitive calls) in 0.645 seconds

**%memit**   
peak memory: 912.58 MiB, increment: 0.41 MiB

**Model Parameters**  
learning_rate = 0.02
steps = 200  

**Linear Regression with Multiple Variable**

In [0]:
feature_columns = ["households", "median_income", "rooms_per_person", "total_rooms", "housing_median_age"]
label = "median_house_value"

In [0]:
X_train = train[feature_columns].values
Y_train = train[label].values.reshape(-1,1)
X_test = test[feature_columns].values
Y_test = test[label].values.reshape(-1,1)

_train = {'x': X_train, 'y': Y_train}
_test = {'x': X_test, 'y': Y_test}

In [ ]:
linear_reg(_train, _test, feature_size=5, epochs=steps, learning_rate=learning_rate)

In [53]:
%memit linear_reg(_train, _test, feature_size=5, epochs=steps, learning_rate=learning_rate)

Training Cost Final: 0.16296034105465065  Testing Cost Final: 0.1645281065761031
peak memory: 912.59 MiB, increment: 0.01 MiB


In [54]:
%time linear_reg(_train, _test, feature_size=5, epochs=steps, learning_rate=learning_rate)

Training Cost Final: 0.17375238247136757  Testing Cost Final: 0.17382207602788013
CPU times: user 678 ms, sys: 65.6 ms, total: 744 ms
Wall time: 673 ms


In [55]:
%prun linear_reg(_train, _test, feature_size=5, epochs=steps, learning_rate=learning_rate)

Training Cost Final: 0.16307964427021374  Testing Cost Final: 0.16496588986564548
 

**Results** 

**Train Loss (RMSE)**: 0.163079  
**Validation Loss(RMSE)**: 0.164965

**%time**    
CPU times: user 678 ms, sys: 65.6 ms, total: 744 ms
Wall time: 673 ms

**%prun**   
 267016 function calls (261206 primitive calls) in 0.796 seconds

**%memit**   
peak memory: 912.59 MiB, increment: 0.01 MiB

**Model Parameters**  
learning_rate = 0.02
steps = 200  

### Neural Network High Level API

**Model Parameters for NN**

In [0]:
# Model Parameters
learning_rate = 0.02
steps = 200
batch_size = 100
hidden_units = [50,100,50]

**Neural Networks** 

In [0]:
feature_columns = ["households", "median_income", "rooms_per_person", "total_rooms", "housing_median_age"]
label = "median_house_value"

# Input functions
train_input = pandas_fn(train, train[label], features=feature_columns, batch_size=batch_size)
test_input = pandas_fn(train, train[label], features=feature_columns, batch_size=1, shuffle=False, num_epochs=1)
predict_input = pandas_fn(test, test[label], features=feature_columns, batch_size=1, shuffle=False, num_epochs=1)

fns = {'train': train_input, 'test': test_input, 'predict': predict_input}
fn_labels = {'train': train[label], 'test': test[label]}

In [ ]:
neural_estimator(fn_labels, fns, feature_columns, learning_rate, steps, hidden_units=hidden_units)

In [59]:
%memit neural_estimator(fn_labels, fns, feature_columns, learning_rate, steps, hidden_units=hidden_units, quiet=True)

Printing Eval Results....
 {'average_loss': 0.02612773, 'label/mean': 0.3967738, 'loss': 0.02612773, 'prediction/mean': 0.44244486, 'global_step': 200}
  Loss for Train: 0.161641
  Loss for Test: 0.162671
peak memory: 942.11 MiB, increment: 10.61 MiB


In [60]:
%time neural_estimator(fn_labels, fns, feature_columns, learning_rate, steps, hidden_units=hidden_units, quiet=True)

Printing Eval Results....
 {'average_loss': 0.023642309, 'label/mean': 0.3967738, 'loss': 0.023642309, 'prediction/mean': 0.39691144, 'global_step': 200}
  Loss for Train: 0.153761
  Loss for Test: 0.155472
CPU times: user 1min 31s, sys: 11.4 s, total: 1min 42s
Wall time: 1min 1s


In [61]:
%prun neural_estimator(fn_labels, fns, feature_columns, learning_rate, steps, hidden_units=hidden_units, quiet=True)

Printing Eval Results....
 {'average_loss': 0.027109774, 'label/mean': 0.3967738, 'loss': 0.027109774, 'prediction/mean': 0.3498774, 'global_step': 200}
  Loss for Train: 0.164650
  Loss for Test: 0.165201
 

**Results**

**Train Loss (RMSE)**: 0.161641  
**Validation Loss(RMSE)**: 0.162671 

**%time**    
CPU times: user 1min 31s, sys: 11.4 s, total: 1min 42s
Wall time: 1min 1s

**%prun**   
7805976 function calls (7479011 primitive calls) in 65.189 seconds

**%memit**   
peak memory: 942.11 MiB, increment: 10.61 MiB

**Model Parameters**  
learning_rate = 0.02  
steps = 100  
batch_size = 100
hidden_units = (50,100,50)

### Neural Networks Low Level API

**Model Parameters for Neural Networks**

In [0]:
# Model Parameters
learning_rate = 0.02
steps = 200
hidden_units=[50,100,50]

**Deep Neural Networks**

In [0]:
# tf.reset_default_graph()

In [0]:
feature_columns = ["households", "median_income", "rooms_per_person", "total_rooms", "housing_median_age"]
label = "median_house_value"

In [0]:
X_train = train[feature_columns].values
Y_train = train[label].values.reshape(-1,1)
X_test = test[feature_columns].values
Y_test = test[label].values.reshape(-1,1)

_train = {'x': X_train, 'y': Y_train}
_test = {'x': X_test, 'y': Y_test}

In [ ]:
neural_network(_train, _test, 5, layer_sizes=hidden_units, epochs=steps, output_size=1, learning_rate=learning_rate)

In [68]:
%memit neural_network(_train, _test, 5, layer_sizes=hidden_units, epochs=steps, output_size=1, learning_rate=learning_rate)

Training Cost Final: 0.14193529067384011  Testing Cost Final: 0.1433433936979341
peak memory: 976.52 MiB, increment: 0.02 MiB


In [70]:
%time neural_network(_train, _test, 5, layer_sizes=hidden_units, epochs=steps, output_size=1, learning_rate=learning_rate)

Training Cost Final: 0.14091385209487187  Testing Cost Final: 0.14319152048295128
CPU times: user 1.09 s, sys: 239 ms, total: 1.33 s
Wall time: 1.31 s


In [72]:
%prun neural_network(_train, _test, 5, layer_sizes=hidden_units, epochs=steps, output_size=1, learning_rate=learning_rate)

Training Cost Final: 0.14604747467717816  Testing Cost Final: 0.14767921158209849
 

**Neural Networks Results**

**Train Loss (RMSE)**: 0.146047  
**Validation Loss(RMSE)**: 0.146792 

**%time**    
CPU times: user 1.09 s, sys: 239 ms, total: 1.33 s
Wall time: 1.31 s

**%prun**   
 366335 function calls (355564 primitive calls) in 1.328 seconds

**%memit**   
peak memory: 976.52 MiB, increment: 0.02 MiB

**Model Parameters**  
learning_rate = 0.02  
steps = 100  
batch_size = 100
hidden_units = (50,100,50)